# 10.1.3

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.linear_model import Perceptron

In [ ]:
# わかりやすいようにDataFrameを作ってみる
iris = load_iris()
x_ = iris.data # 一旦全指定
y_ = (iris.target).astype(np.int) # 一旦全指定

In [ ]:
import pandas as pd
df_ = pd.DataFrame(x_)
df_.columns = ['sepal length', 'sepal width', 'petal length', 'petal width']
df_['target'] = y_

'''columns'''
# ‘sepal length (cm)’　萼の長さ：0
# ‘sepal width (cm)’　萼の幅:1
# ‘petal length (cm)’　花弁の長さ:2
# ‘petal width (cm)’　花弁の幅:3

'''target'''
# setosa：0
# versicolor:1
# virginica:2

df_

In [ ]:
x = iris.data[:, (2, 3)]
y = (iris.target==0).astype(np.int)
# setosaかどうか分類するモデルを作成

In [ ]:
df = pd.DataFrame(x)
df.columns = ['petal length', 'petal width']
df['target'] = y
df

In [ ]:
per_clf = Perceptron()
per_clf.fit(x, y)
y_pred = per_clf.predict([[1, 0.5]])
y_pred

# 10.1.4

In [ ]:
# ステップ関数
import matplotlib.pyplot as plt
x = np.linspace(-7,7)

step = 1.0*(x>=0)
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot()
ax.plot(x, step)
plt.show()

In [ ]:
# ロジスティック関数(シグモイド関数)
sigmoid = 1/(1+np.exp(-x))
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot()
ax.plot(x, sigmoid)
plt.show()

In [ ]:
# ハイパボリックタンジェント
tanh = np.tanh(x)
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot()
ax.plot(x, tanh)
plt.show()

In [ ]:
relu = x*(x>=0)
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot()
ax.plot(x, relu)
plt.show()

# 10.1.5

In [ ]:
softplus = np.log(1 + np.exp(x))
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot()
ax.plot(x, softplus)
plt.show()

# 10.1.6 

In [ ]:
# 分類で使う場合の交差エントロピー誤差
y = np.linspace(0, 1)
cross = -np.log(y)
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot()
ax.plot(x, cross)
plt.show()

# 10.2.2.1-10.2.2.2

In [ ]:
import tensorflow as tf
from tensorflow import keras
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)
print(tf.__version__)
print(keras.__version__)

In [ ]:
# Datasetのロード
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

In [ ]:
print(X_train_full.shape)
print(X_train_full.dtype)
print(X_test.shape)

In [ ]:
X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.0

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

class_names[y_train[0]]

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

# 下記のような記載方法もある
# model = keras.models.Sequential([
#     keras.layers.Flatten(input_shape=[28, 28]),
#     keras.layers.Dense(300, activation='relu'),
#     keras.layers.Dense(100, activation='relu'),
#     keras.layers.Dense(10, activation='softmax')
#     ])

In [ ]:
# keras.ioについて

from sklearn.model_selection import train_test_split

import sklearn.model_selection
sklearn.model_selection.train_test_split

import sklearn
sklearn.model_selection.train_test_split

In [ ]:
# modelの中身を確認
model.summary()

In [ ]:
# モデルの層をリストで確認
model.layers

In [ ]:
hidden1 = model.layers[1]

In [ ]:
hidden1.name

In [ ]:
model.get_layer('dense') is hidden1

In [ ]:
weights, biases = hidden1.get_weights()
print(f'weight size: {weights.shape}')
print(f'bias size: {biases.shape}')

In [ ]:
weights

In [ ]:
biases

# 10.2.2.3

In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

#### パラメータの詳細について → https://keras.io/api/losses/

#### sparse_categorical_crossentropyとcategorical_crossentropyの違い　→ https://qiita.com/yy51/items/2f1d2651fec3b19770e0

#### オプティマイザについて　→ https://qiita.com/omiita/items/1735c1d048fe5f611f80

# 10.2.2.4

In [ ]:
X_valid.shape

In [ ]:
history = model.fit(X_train, y_train, epochs=30, validation_data=(X_valid, y_valid))

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [ ]:
history.params

In [ ]:
history.epoch

In [ ]:
history.history

#### 精度改善のためのパラメータ調整はまずは「学習率」と「オプティマイザ」を変更してみる
#### その後に層の数、ニューロンの数、活性化関数など、構造をいじる

In [ ]:
model.evaluate(X_test, y_test)

# 10.2.2.5

In [ ]:
X_new = X_test[:3]
y_proba = model.predict(X_new)
y_proba.round(2)

In [ ]:
y_pred = np.argmax(model.predict(X_new), axis=-1)
np.array(class_names)[y_pred]